In [3]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import torch
import os
import tqdm
%matplotlib inline

In [6]:
class ObjectDetector:
    def __init__(self, model_path=None):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f'device = {self.device}')
        if model_path is None:
            self.model = YOLO('yolov8m.pt').to(self.device)
        else:
            self.model = YOLO(model_path).to(self.device)
            
    def predict_image(self, image_path):
        return self.model(image_path)
    
    def predict_dir(self, dir_path):
        return self.model(dir_path)
    @staticmethod
    def get_image_metadata(result, cnt, dataset_id=1):
        image_metadata = {}
        height, width = result.orig_shape
        image_metadata['id'] = cnt
        image_metadata['dataset_id'] = dataset_id
        image_metadata['path'] = result.path
        image_metadata['height'], image_metadata['width'] = height, width
        image_metadata['file_name'] = os.path.basename(result.path)
        return image_metadata

        
        
    def coco_annotate(self, dir_path):
        results = self.predict_dir(dir_path)
        coco_output = {
            "images": [],
            "categories": [
                {"id": 1, "name": "Elephant", "supercategory": "", "color": "#3ab7dd", "metadata": {}, "keypoint_colors": []}
            ],
            "annotations": []
        }
        annotation_id = 1
        for i, result in enumerate(results):
            print(result)
            image_metadata = self.get_image_metadata(result, i)
            
            
    def show(self, img):
        plt.figure()
        plt.axis('off')
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) 
        plt.show()
    def show_labeled(self, img_path):
        result = self.predict_image(img_path)
        labeled_img = result[0].plot()
        self.show(labeled_img)

        
        
object_detector = ObjectDetector('best.pt')
object_detector.coco_annotate('cx')

device = cpu

image 1/4 /Users/tomasnguyen/FIT/BP/cx/frame_2.jpg: 384x640 3 Elephants, 1223.1ms
image 2/4 /Users/tomasnguyen/FIT/BP/cx/frame_6.jpg: 384x640 1 Elephant, 1448.1ms
image 3/4 /Users/tomasnguyen/FIT/BP/cx/frame_8.jpg: 384x640 (no detections), 1284.8ms
image 4/4 /Users/tomasnguyen/FIT/BP/cx/frame_9.jpg: 384x640 (no detections), 1372.5ms
Speed: 6.9ms preprocess, 1332.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)


100%|██████████| 4/4 [00:00<00:00, 409.50it/s]

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Elephant'}
obb: None
orig_img: array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [ 28,  26,  15],
        [ 74,  72,  62],
        [  5,   2,   0]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [ 27,  25,  14],
        [ 76,  74,  64],
        [  6,   3,   0]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [ 29,  25,  14],
        [ 78,  72,  65],
        [  7,   1,   0]],

       ...,

       [[ 50,  64,  52],
        [ 50,  64,  52],
        [ 50,  64,  52],
        ...,
        [ 21,  22,  18],
        [ 18,  19,  15],
        [ 17,  18,  14]],

       [[ 50,  64,  52],
        [ 50,  64,  52],
        [ 50,  64,  52],
        ...,
        [ 20,  21,  17],
        [ 18,  19,  15],
        [ 1